In [1]:
!pip install pykalman

     |████████████████████████████████| 235kB 7.5MB/s 
  Created wheel for pykalman: filename=pykalman-0.9.5-cp36-none-any.whl size=48464 sha256=1d7ad74f9e8f20f0ff297da51661e2c72da07cd089ebe0d170d0fbc6a05f0877
  Stored in directory: /root/.cache/pip/wheels/d9/e8/6a/553d9832679cb74a8434fa597c3abdb07313e40054a0adf9ac
Successfully built pykalman


In [8]:
!unzip data.zip

Archive:  data.zip
   creating: data/processed/
   creating: data/raw/
   creating: data/raw/1/
  inflating: data/raw/1/p1-ankle-1.csv  
  inflating: data/raw/1/p1-ankle-2.csv  
  inflating: data/raw/1/p1-ankle-3.csv  
  inflating: data/raw/1/p1-hand-1.csv  
  inflating: data/raw/1/p1-hand-2.csv  
  inflating: data/raw/1/p1-hand-3.csv  
  inflating: data/raw/1/p1-pocket-1.csv  
  inflating: data/raw/1/p1-pocket-2.csv  
  inflating: data/raw/1/p1-pocket-3.csv  
   creating: data/raw/2/
  inflating: data/raw/2/p2-ankle-1.csv  
  inflating: data/raw/2/p2-ankle-2.csv  
  inflating: data/raw/2/p2-ankle-3.csv  
  inflating: data/raw/2/p2-hand-1.csv  
  inflating: data/raw/2/p2-hand-2.csv  
  inflating: data/raw/2/p2-hand-3.csv  
  inflating: data/raw/2/p2-pocket-1.csv  
  inflating: data/raw/2/p2-pocket-2.csv  
  inflating: data/raw/2/p2-pocket-3.csv  
   creating: data/raw/3-left/
  inflating: data/raw/3-left/Jordan_ankle_left_1.csv  
  inflating: data/raw/3-left/Jordan_ankle_left_2.csv  
 

In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
from datetime import datetime
import glob
# from statsmodels.nonparametric.smoothers_lowess import lowess

In [3]:
def to_float(x):
    return float(x)

In [4]:
def get_csv(filename):
    column_name = ['time', 'x_accel', 'y_accel', 'z_accel']
    raw_data = pd.read_csv(filename, names=column_name, header=None) 
    raw_data = raw_data.drop(0)
    raw_data['time'] = raw_data['time'].apply(to_timestamp)
    raw_data['time'] = raw_data['time']-raw_data['time'][1]
    raw_data['x_accel'] = raw_data['x_accel'].apply(to_float)
    raw_data['y_accel'] = raw_data['y_accel'].apply(to_float)
    raw_data['z_accel'] = raw_data['z_accel'].apply(to_float)
    
    return raw_data

In [5]:
def to_timestamp(x):
    x = datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')
    return datetime.timestamp(x)

In [27]:
def extract_features(raw_data):
  raw_data = raw_data[raw_data['time'] < 30]
  mean = raw_data.mean().tolist()
  std = raw_data.std().tolist()
  return mean, std

In [30]:
def process_data(input_file_dir, output_dir ,tester_num):
  mean_frames = []
  std_frames = []
  fileNames = sorted(glob.glob(input_file_dir + '*'))
  for name in fileNames:
    raw_data = get_csv(name)
    mean, std = extract_features(raw_data)
    mean_frames.append(mean)
    std_frames.append(std)

  output_file = output_dir + 'p' + tester_num
  col = ['time', 'x_accel', 'y_accel', 'z_accel']
  results_mean = pd.DataFrame(mean_frames, columns=col)
  results_std = pd.DataFrame(std_frames, columns=col)
  results_mean['tester'] = 3
  results_std['tester'] = 3

  # output to .csv files.
  results_mean.to_csv(output_file + "-mean.csv")
  results_std.to_csv(output_file + "-std.csv")

In [33]:
file_loc_left = "data/raw/3-left/"
file_loc_right = "data/raw/3-right/"

output_dir = "data/processed/"
output_dir_right = "data/processed/right/"

process_data(file_loc_left, output_dir, '3')

# Saving a second csv to see if we can accuratly predict which side the phone is on
process_data(file_loc_right, output_dir_right, '3')
